In [0]:
%scala

val storageAccount = "ppimentelh" //nombre de la cuenta de almacenamiento
val container = "ppimentelh" //nombre del contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:34:58Z&st=2024-12-03T00:34:58Z&spr=https&sig=EVibf7Zg4nmsHywTt3rWcE6rhNk43nyxdwxF2N34Cnc%3D" //llave de conexion

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

//Crear acceso directo
dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = ppimentelh
container: String = ppimentelh
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:34:58Z&st=2024-12-03T00:34:58Z&spr=https&sig=EVibf7Zg4nmsHywTt3rWcE6rhNk43nyxdwxF2N34Cnc%3D
conf: String = fs.azure.sas.ppimentelh.ppimentelh.blob.core.windows.net
res2: Boolean = true

In [0]:
## INGESTAR ARCHIVO PRODUCTO.PARQUET
producto = (spark.read.parquet("/mnt/ppimentelh/proyectofinal/capaplata/Producto.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
## INGESTAR ARCHIVO CATEGORIA.PARQUET
categoria = (spark.read.parquet("/mnt/ppimentelh/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
## INGESTAR ARCHIVO SUBCATEGORIA.PARQUET
subcategoria = (spark.read.parquet("/mnt/ppimentelh/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql

--Eliminar tabla en caso exista
DROP TABLE IF EXISTS Fact_Product;

--Crear tabla
CREATE TEMP VIEW Fact_Product
AS
SELECT P.Cod_Producto, P.Nombre AS Producto, S.Nombre AS SubCategoria, C.Nombre AS Categoria
FROM s_producto P
JOIN s_subcategoria S ON P.Cod_SubCategoria = S.Cod_SubCategoria
JOIN s_categoria C ON S.Cod_Categoria = C.Cod_Categoria 

In [0]:
## RESULTADO SE LLEVA A CAPA ORO
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/ppimentelh/proyectofinal/capaoro/FactProducto.parquet")

In [0]:
#Eliminar acceso directo
dbutils.fs.unmount("/mnt/ppimentelh")

/mnt/ppimentelh has been unmounted.


True